In [1]:
import pandas as pd
import requests
from pypdf import PdfReader
from io import BytesIO         
import tabula        
import re

In [2]:
df = pd.read_excel("../Base 5.1.xlsx")
df

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo
0,Adriana Bruscato Bortoluzzo,Dados,https://periodicos.fgv.br/rbfin/article/view/8...,2021,Artigo Acadêmico,Acadêmico,"adriana bruscato bortoluzzo, pistil, gustavo n...",Site,text/html,False
1,Paulo Furquim de Azevedo,Metricis,https://repositorio.insper.edu.br/entities/pub...,2016,Capítulo de Livro,Livros,Paulo Furquim,Bloqueado,text/html; charset=utf-8,False
2,Sérgio Lazzarini,Metricis,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,2017,Artigo Acadêmico,Acadêmico,sergio giovanetti lazzarini,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,PDF,False
3,Marcelo Marchesini,Metricis,https://www.tandfonline.com/doi/abs/10.1080/09...,2019,Artigo Acadêmico,Acadêmico,marcelo marchesini da costa,Site,text/html; charset=utf-8,False
4,Leticia Faria de Carvalho Nunes,Saúde,https://repositorio.insper.edu.br/entities/pub...,2021,Working Paper,Pesquisa,leticia faria de carvalho nunes,https://repositorio.insper.edu.br/bitstreams/5...,text/html; charset=utf-8,True
...,...,...,...,...,...,...,...,...,...,...
760,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://www.portaldeperiodicos.idp.edu.br/dire...,2020,Artigo Acadêmico,Acadêmico,NaN,Site,text/html; charset=utf-8,False
761,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2020,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
762,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2023,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
763,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2022,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False


In [3]:
# Pegando os índices das células que contém "Site" na coluna "PDF"
block_and_no_resume = []
for i in range(len(df["PDF"])):
    if df.loc[i, "PDF"] == "Bloqueado":
        if df.loc[i, "Resumo"] == False:
            block_and_no_resume.append("fudeu")
        else: 
            block_and_no_resume.append("da pra sonhar")
    else:
        block_and_no_resume.append("ok")

print(block_and_no_resume)

['ok', 'fudeu', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'da pra sonhar', 'ok', 'ok', 'ok', 'ok', 'da pra sonhar', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'fudeu', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'da pra sonhar', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'fudeu', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'fudeu', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 

In [6]:
count = 0
for i in block_and_no_resume:
    if i == "fudeu":
        count += 1

print(count)

34


In [ ]:
# lógica para pegar um pdf

url = "https://repositorio-api.insper.edu.br/server/api/core/bitstreams/51e25ba0-62a1-45e4-be5a-add48cdbcf52/content"


response = requests.get(url)
response.raise_for_status() 

pdf_file = BytesIO(response.content)
reader = PdfReader(pdf_file)

texto = ""
for i in range(0,6):
    page = reader.pages[i]
    texto_formatado = "".join(page.extract_text())
    texto += texto_formatado

print(texto)

Cobertura Vacinal no Brasil
2020
Letícia Nunes
Instituto de Estudos para Políticas de Saúde
Maio, 2021Sumário
1 Visão Geral 3
2 Poliomielite 7
3 Tríplice Viral 13
4 BCG 20
5 Hepatite B em Crianças até 30 dias 27
6 Hepatite A 33
7 Pneumocócica 39
8 Meningococo C 45
9 Rotavírus Humano 51
10 Pentavalente 57
Instituto de Estudos para Políticas de Saúde
Panorama da Cobertura Vacinal no Brasil, 2020
Maio de 2021
Rua Itapeva 286 | 81-84
Bela Vista, São Paulo – SP
01332-000 – Brasil
www.ieps.org.br
+55 11 4550-2556
contato@ieps.org.br1 Visão Geral
A vacinação foi eleita sucessivamente uma das dez maiores conquistas da saúde pública no último
século, sendo responsável por uma incrível redução da morbidade e mortalidade por doenças
infecciosas em todo o mundo (CDC 1999, 2011). Estimativas revelam que vacinas administradas
entre 2000 e 2030 em países de baixa e média renda podem prevenir 69 milhões de mortes, princi-
palmente entre crianças de até 5 anos (Li et al. 2021). E, nesse último ano, a p